# Centrality

Centrality is a central metric in network analysis used to determine the importance of a node within a network based on its structural position. It can also be applied to individual edges, measuring the significance of a link within the network. Numerous centrality metrics have been developed, and in this assignment, we will explore some commonly used centrality metrics.

#### Network data

We will use the worldwide airport network based on data taken from Openflight.org.

In [ ]:
import pandas as pd
import igraph
import numpy as np

node_table = pd.read_csv(
    "https://raw.githubusercontent.com/skojaku/adv-net-sci-course/main/data/airport_network_v2/node_table.csv"
)
edge_table = pd.read_csv(
    "https://raw.githubusercontent.com/skojaku/adv-net-sci-course/main/data/airport_network_v2/edge_table.csv"
)
src, trg = tuple(edge_table[["src", "trg"]].values.T)
edge_list = tuple(zip(src, trg))

# node_id and name dictionary
n_nodes = node_table.shape[0]
id2name = np.array([""] * n_nodes, dtype="<U64")
id2name[node_table["node_id"]] = node_table["Name"].values

g = igraph.Graph(
    edge_list,
    vertex_attrs=dict(Name=id2name, node_id=node_table["node_id"].values),
)

# You can retrieve the airport names by
print(g.vs[0]["Name"], ",", g.vs[1]["Name"], ", ...")

## Degree centrality

The simplest metric of centrality is degree centrality. The centrality of a node is the degree of the node. And with igraph, you can get the degree of nodes by

In [ ]:
degree_centrality = g.degree()

## Path-based centrality

#### Closeness centrality

One way to measure the importance of nodes is to measure how central they are in the network. A node at the center of the network should be able to reach any other nodes in *short* distances. One way to quantify this is to take the sum of the distances from individual nodes to all other nodes in the network, with a simple inversion to make "central" nodes having a high centrality, i.e.,
$$
c_i:= \frac{n-1}{\sum_{j, i\neq j} d(i,j)},
$$
where $d(i,j)$ is the shortest path length between nodes $i$ and $j$, and $n$ is the number of nodes in the network. This metric is called *closeness centrality*. The numerator serves as a scaling constant to make the metric vary in the range $(0,1]$. With `igraph`, it can be computed by


In [ ]:
closeness_centrality = g.closeness()

#### Betweenness centrality

Betweenness centrality is another widespread centrality based on the shortest path lengths. It is defined by
$$
b_i:= \sum_{s \neq t\neq i} \frac{\sigma_{s,i,t}}{\sigma_{s,t}}
$$
where $\sigma_{s,t}$ is the number of shortest paths between node $s$ and node $v$, and $\sigma_{s,i,t}$ is the shortest paths between $s$ and $t$ that goes through node $i$. A node with a high betweenness centrality means that the node is a dominant intermediary of flows (that go through the shortest path) between nodes on the network. Or, you can think of it as a bottleneck of flows. With igraph, you can compute the betweenness centrality by

In [ ]:
betweenness_centrality = g.closeness()

The idea of betweenness centrality can be extended to edges. Instead of counting the number of shortest paths going through nodes, we can count them for each edge and define the betweenness centrality in the same way. You can compute the edge betweenness centrality with igraph by

In [ ]:
edge_betweeness_centrality = g.edge_betweenness()  # compute the betweenness centrality

# You can retrieve the edge lists pertaining to the edge betweenness centrality by
edge_list = g.get_edgelist()

# For example, the betweenness centrality edge_betweeness_centrality[10] corresponds to an edge edge_list[10]

# Robustness

Another approach to determining the importance of nodes is by evaluating the impact their removal would have on the network. For instance, an individual can be deemed important within a social network if their departure leads to the network becoming fragmented into disconnected segments. Similarly, certain Internet routers are considered critical to the functioning of the Internet, as their removal would result in the malfunctioning of systems.

We can consider two types of network damage: random failure and target attack. In a random failure scenario, nodes are selected at random for removal. However, in a targeted attack, nodes are strategically chosen for removal based on specific criteria. For instance, nodes may be removed in the descending order of their degree centrality.


## Random failure

Let's break the network 😈! Before damaging the network, we should damage a copy of the network while keeping the original network as it is. This way, we can come back to the original network and compare it against the damaged one.

In [ ]:
g_damaged = g.copy()

To simulate random failures, we randomly sample some nodes and remove them from the network.

In [ ]:
# Select the number of nodes to be removed
n_nodes_removed = 10
nodes_to_be_removed = np.random.choice(n_nodes, size=n_nodes_removed, replace=False)

# Remove them from the network
g_damaged.delete_vertices(nodes_to_be_removed)

If the removal of the nodes is substantial to the network, we expect to see the network to be fragmented into small connected components. To assess the connectivity of the network, one can measure the fraction of nodes in the giant component in all nodes in the network.

In [ ]:
def eval_connectivity(g_damaged, g):
    # If the damaged network has no node left, return 0 connectivity
    if g_damaged.vcount() == 0:
        return 0.0

    # Find the connected components
    component_membership = g_damaged.connected_components().membership

    # Count the number of nodes in each component
    n_nodes_comp = np.bincount(component_membership)

    # Return the proportion of nodes in the giant component
    return np.max(n_nodes_comp) / float(g.vcount())


score_orig = eval_connectivity(g, g)
score_damaged = eval_connectivity(g_damaged, g)
print(score_orig, score_damaged)
# n_connected_comp = len(g.components())
# n_connected_comp_damaged = len(g_damaged.components())

A network is robust if the network does not break into small pieces if some nodes are removed. Let's test the robustness more systematically. We iteratively remove some number of nodes and see how fast/slow network connectivity drops. More specifically,

In [ ]:
g_damaged = g.copy()

n_nodes_removed = 1
n_nodes = g.vcount()  # Number of nodes

results = []
while g_damaged.vcount() > 0:  # Loop if the network has at least one nod
    # Get the nodes in the network
    nodes_in_networks = g_damaged.vs.indices

    # Sample nodes to remove
    nodes_removed = np.random.choice(
        nodes_in_networks, size=np.minimum(n_nodes_removed, len(nodes_in_networks))
    )

    # Delete
    g_damaged.delete_vertices(nodes_removed)

    # Evaluate the network connectivity
    score = eval_connectivity(g_damaged, g)

    # Save the results
    results += [
        {
            "score": score,
            "frac_nodes_removed": 1 - g_damaged.vcount() / n_nodes,
            "attack": "random",
        }
    ]

plot_data = pd.DataFrame(results)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("white")
sns.set(font_scale=1.2)
sns.set_style("ticks")

fig, ax = plt.subplots(figsize=(5, 5))
ax = sns.lineplot(data=plot_data, x="frac_nodes_removed", y="score")
ax.set_xlabel("Proportion of nodes removed")
ax.set_ylabel("Proportion of nodes in a core")
sns.despine()

### Targeted attack

In a targeted attack, we will remove nodes based on some criteria. Let us demonstrate the idea by using the degree centrality. With the degree centrality, we will remove nodes with the highest degree in the network.

In [ ]:
g_damaged = g.copy()

n_nodes_removed = 1
n_nodes = g.vcount()  # Number of nodes

while g_damaged.vcount() > 0:  # Loop if the network has at least one nod
    # Get the nodes in the network
    nodes_in_networks = np.array(g_damaged.vs.indices)

    # Get the degree centrality
    deg = np.array(g_damaged.degree())

    # Find the nodes with the highest degree.
    nodes_removed = np.argsort(deg)[::-1][
        : np.minimum(n_nodes_removed, len(nodes_in_networks))
    ]

    # Delete
    g_damaged.delete_vertices(nodes_removed)

    # Evaluate the network connectivity
    score = eval_connectivity(g_damaged, g)

    # Save the results
    results += [
        {
            "score": score,
            "frac_nodes_removed": 1 - g_damaged.vcount() / n_nodes,
            "attack": "targeted",
        }
    ]

plot_data = pd.DataFrame(results)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("white")
sns.set(font_scale=1.2)
sns.set_style("ticks")

fig, ax = plt.subplots(figsize=(5, 5))
ax = sns.lineplot(data=plot_data, x="frac_nodes_removed", y="score", hue="attack")
ax.set_xlabel("Number of nodes removed")
ax.set_ylabel("Proportion of nodes in a core")
ax.legend(frameon=False)
sns.despine()

The robustness profile curve provides insights into the network's resilience against random attacks and random closure of airports. While the network demonstrates robustness to such events, it is important to note that removing a small number of airports, particularly those with a high degree, can significantly impact the network.

While the profile curve is informative, it is convenient to summarize the curve into a single number, called *$R$-index*. It is defined as the area under the robustness curve.

$$
R = \frac{1}{N} \sum_{x=1}^N y(x/N)
$$
where $y$ is the robustness at fraction $x/N$, and $N$ is the number of all nodes in the network. A higher value indicates that the network is robust against the attack. The $R$-index has a maximum value of 1/2 (i.e., which corresponds to a diagonal line in the plot above).

In [ ]:
plot_data.groupby("attack").mean()[["score"]]

# Or alternatively
for attack_type, df in plot_data.groupby("attack"):
    rindex = df["score"].sum() / n_nodes
    print(attack_type, "%.3f" % rindex)

# Core Decomposition

![](https://www.researchgate.net/publication/337008832/figure/fig1/AS:834053125206017@1575865169184/Example-of-the-k-core-decomposition.png)

What if we remove nodes from a network in reverse order, starting with the small-degree nodes and moving toward the high-degree nodes? This approach, known as *$k$-core decomposition*, aims to iteratively remove unimportant nodes to identify the important nodes that survive throughout the removal process.

The $k$-core is the maximal subgraph of a network consisting of nodes with degrees greater than or equal to $k$.
There is a simple algorithm to identify them, which operates as follows:

1. Calculate the degree of nodes in the network
2. Remove the nodes with degree $k$ or less in the network.
3. Recalculate the degree
4. If all nodes have a degree less than $k$ in the removed network, terminate the algorithm. Otherwise, go back to step 2.

Let's implement the algorithm.


In [ ]:
g_core = g.copy()
k = 20
while True:
    # Find nodes in the network
    nodes_in_network = np.array(g_core.vs.indices)

    # Calculate the degree
    degree = np.array(g_core.degree())

    # Find nodes with degree $k$ or less
    nodes_to_remove = nodes_in_network[degree < k]

    # If there is no node to remove, then terminate the algorithm.
    if len(nodes_to_remove) == 0:
        break

    # Remove
    g_core.delete_vertices(nodes_to_remove)

In [ ]:
core_node_ids = g_core.vs()["node_id"]

core_node_table = node_table.iloc[core_node_ids]
labels = core_node_table["Name"].values

node_color = core_node_table["region"].values
palette = sns.color_palette("colorblind")
cmap = {k: palette[i] for i, k in enumerate(set(node_color))}
node_color = core_node_table["region"].map(cmap)

igraph.plot(
    g_core,
    vertex_label=labels,
    vertex_size=10,
    edge_width=0.1,
    vertex_color=node_color,
    layout="kk",
    bbox=(500, 500),
    vertex_label_size=10,
)

$k$-core has many interesting properties.
1. $k$-core is nested, meaning that $k$ core contains $k+1$ core.
2. $k$-core sets the upper bounds for the density of edges and the diameter of the subgraph.
3. $k$-core contains the largest cliques.

$k$-core number of a node is the maximum $k$ that the node survives the node removal process. For instance, if a node has $k$-core number of 3, it means that it belongs to $1,2,3$-cores but does not belong to $4$-core and above. Thus, nodes with a high-core number are those that can be regarded as the important nodes in the network.

With igraph, it is easy to find the $k$-core number, with a more efficient algorithm. So, we don't need to actually write the code to get it. But I hope you find it fun to see how it is identified.

In [ ]:
kcore_number = g.coreness()

# The above core can be generated by
# g_cores = g.induced_subgraph(np.where(np.array(kcore_number)>=k)[0])

Some useful tips about $k$-core decomposition.

- $k$-core decomposition is a quite useful tool in practice to downsize the network. Many empirical networks are notoriously large, and it is often impossible to analyze with a given computing capacity at hand. A remedy is to focus on the $k$-core of the network since most edges are held by the nodes in the large-degree nodes. Since most nodes have a small degree in many empirical networks, using a moderately small $k$ can drastically reduce the size of the network.
- The concept of the $k$-core is extended to the directed network: https://link.springer.com/article/10.1007/s10115-012-0539-0




# Assignments


**Question 1: Which airport is the most central in terms of the closeness centrality? List the top 5 most central airports. You may use the APIs to compute the shortest path, but do not use built-in APIs that give the closeness centrality such as igraph.Graph.closeness**.

In [ ]:
# Your code -------------------
# Compute the centrality

centrality = np.zeros(n_nodes)
centrality = (
    ...
)  # Compute the closeness centrality of each node $i$ and set it to centrality[i].
# -------------------

# Check the correctness of the computed closeness
assert np.all(np.isclose(centrality, g.closeness()))

# Show the top K airport with the largst centrality
topK = 5
top_airport_ids = np.argsort(centrality)[::-1][:topK]
node_table.iloc[top_airport_ids][["Name"]]

**Question 2: Compute the betweenness centrality of the airports and show the top 5 airports. You may use APIs that directly compute the betweenness centrality, such as igraph.Graph.betweenness**.

You should see an interesting airport that is not central in terms of the closeness centrality but is central in terms of the betweenness centrality. If you are curious why, here is a nice reading to understand why: https://toreopsahl.com/2011/08/12/why-anchorage-is-not-that-important-binary-ties-and-sample-selection/

**Question 3: Plot the robustness profile curve for the betweenness centrality, together with the curves for random attack and the target attack by the degree centrality. Compute the $R$-index for the betweenness centrality.**

**Question 4: Identify the airports with the highest k-core number and plot their subgraph**

In [ ]:
# Your code ------

# Generate an igraph object g_core that consists of the nodes in the k-core of the largest k
g_core = ...
# ----------------

# Run the following code to help us grade
assert g_core.vcount() == 38